# Introduction to QP

**Learning Objectives**
- Understand the basics of Drake's MathematicalProgram
- Learn how to formulate and solve a quadratic program (QP) in Drake

**What You'll Implement**
- Solving a simple QP in Drake

## Setup and Imports

Let us first import our Mathematical Programming tools frome Drake

In [38]:
# python libraries
import numpy as np
from pydrake.all import MathematicalProgram, Solve, eq, ge, le

## Part 1: Introduction to MathematicalProgram 

The purpose of this exercise is to get you familiar with the basics of what an instance of an optimization problem is, as well as how to solve it. 

An optimization problem is usually written as 

$$\begin{aligned} \min_x \quad & f(x) \\ \textrm{s.t.} \quad & g(x)\leq 0,\\ \quad &  h(x)=0 \end{aligned}$$

We call $x$ the **decision variable**, $f(x)$ the **cost function**, $g(x)\leq 0$ an **inequality constraint**, and $h(x)=0$ an **equality constraint**. We usually denote the optimal solution by $x^*$. Most of the times, the constraints are hard-constraints, meaning that they must be fulfilled by the optimal solution. 

Drake offers a very good interface to many solvers using `MathematicalProgram`. Let's try to solve a simple problem using `MathematicalProgram`: 

$$\begin{aligned} \min_x \quad & \frac{1}{2}x^2 \\ \textrm{s.t.} \quad & x\geq 3 \end{aligned}$$

Before we start coding, what do you expect the answer to be? You should persuade yourself that the optimal solution is $x^*=3$, since that is value at which minimum cost is achieved without violating the constraint.



In [2]:
"""
Steps to solve a optimization problem using Drake's MathematicalProgram
"""

# 1. Define an instance of MathematicalProgram
prog = MathematicalProgram()

# 2. Add decision varaibles
x = prog.NewContinuousVariables(1)

# 3. Add Cost function
prog.AddCost(x.dot(x))

# 4. Add Constraints
prog.AddConstraint(x[0] >= 3)

# 5. Solve the problem
result = Solve(prog)

# 6. Get the solution
if result.is_success():
    print("Solution: " + str(result.GetSolution()))

Solution: [3.]


You should have seen that we were successful in getting the expected solution of $x^*=3$. 

A particular class of problems that we want to focus on this problem are [Quadratic Programs (QP)](https://en.wikipedia.org/wiki/Quadratic_programming), which can be solved very efficiently in practice (even on the order of kHz).

The general formulation of these problems are defined as follows. 

$$\begin{aligned} \min_x \quad & \frac{1}{2}x^T\mathbf{Q}x + c^Tx \\ \textrm{s.t.} \quad & \mathbf{A}x\leq b,\\ \quad &  \mathbf{A}'x=b' \end{aligned}$$

where $\mathbf{Q}$ is a positive-definite, symmetric matrix. Note that the cost is a quadratic function of the decision variables, while the constraints are all linear. This is what defines a convex QP. 

Let's practice solving a simple QP: 

$$\begin{aligned} \min_{x_0,x_1,x_2} \quad & x_0^2 + x_1^2 + x_2^2 \\ \textrm{s.t.} \quad & \begin{pmatrix} 2 & 3 & 1 \\ 5 & 1 & 0 \end{pmatrix} \begin{pmatrix} x_0 \\ x_1 \\ x_2 \end{pmatrix} = \begin{pmatrix} 1 \\ 1 \end{pmatrix}\\  \quad &  \begin{pmatrix} x_0 \\ x_1 \\ x_2 \end{pmatrix} \leq \begin{pmatrix} 2 \\ 2 \\ 2\end{pmatrix} \end{aligned}$$

To conveniently write down constraints that are vector-valued, Drake offers `eq,le,ge` for elementwise constraints. It might take some time to learn the syntax of constraints. For a more well-written and in-depth introduction to `MathematicalProgram`, [this notebook tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/mathematical_program-4c4f4de7d5d3452daacf45c20b315d31) is incredibly useful. 
 


In [39]:
prog = MathematicalProgram()

x = prog.NewContinuousVariables(3)

prog.AddCost(x.dot(x))
prog.AddConstraint(eq(np.array([[2, 3, 1], [5, 1, 0]]).dot(x), [1, 1]))
prog.AddConstraint(le(x, 2 * np.ones(3)))

result = Solve(prog)

# 6. Get the solution
if result.is_success():
    print("Solution: " + str(result.GetSolution()))

Solution: [0.15897436 0.20512821 0.06666667]



## Part 2: Your Turn! 

**YOUR TASK**: You must solve the following problem and store the result in a variable named `result_submission`. 

$$\begin{aligned} \min_{x_0,x_1,x_2} \quad & 2x_0^2 + x_1^2 + 4x_2^2 \\ \textrm{s.t.} \quad & \begin{pmatrix} 1 & 2 & 3 \\ 2 & 7 & 4 \end{pmatrix} \begin{pmatrix} x_0 \\ x_1  \\ x_2 \end{pmatrix} = \begin{pmatrix} 1 \\ 1 \end{pmatrix} \\ \quad &  |x| \leq \begin{pmatrix} 0.35 \\ 0.35 \\ 0.35\end{pmatrix} \end{aligned}$$

NOTE: The last constraint says that the absolute value of `x[i]` must be less than the value of `b_bb[i]`. You cannot put an absolute value directly as a constraint, so there are two routes that you can take:
- Break the constraints down to two constraints that don't involve the absolute value.  
- Drake offers [`AddBoundingBoxConstraint`](https://drake.mit.edu/pydrake/pydrake.solvers.html?highlight=addboundingboxconstraint#pydrake.solvers.MathematicalProgram.AddBoundingBoxConstraint) which you may use in your implementation.

In [46]:
prog = MathematicalProgram()

# TODO: Implement the program here

x = prog.NewContinuousVariables(3)
x_q = [x[0] * 2, x[1] * 1, x[2] * 4]

prog.AddCost(x.dot(x_q))
prog.AddConstraint(eq(np.array([[1, 2, 3], [2, 7, 4]]).dot(x), [1, 1]))
prog.AddBoundingBoxConstraint(-.35 * np.ones(3), .35 * np.ones(3), x)

result = Solve(prog)

# 6. Get the solution
if result.is_success():
    print("Solution: " + str(result.GetSolution()))

result_submission = result.GetSolution()  # store the result here.

Solution: [ 0.15343917 -0.1005291   0.34920635]


Below is our autograder where you can check the correctness of your implementation.

In [47]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_simple_qp import TestSimpleQP

Grader.grade_output([TestSimpleQP], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 4/4.

Score for Test Simple QP problem is 4/4.


## VERIFICATION IN GRADESCOPE

**Prerequisites:** You must complete ALL the TODOs above before these verification exercises will work!

**Instructions:** Implement the exercises below and upload your solutions to Gradescope



## Verification 1: QP Solution

**Question:** What is the solution to the optimization problem you implemented? Put your answer in gradescope. Give your answer to 4 decimal places.